In [8]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import mean_absolute_error, mean_squared_error
from IPython.display import display

# Data Pemakaian Listrik Mingguan dan Faktor Eksogen
data = {
    'Minggu': ['Week 1', 'Week 2', 'Week 3', 'Week 4',
               'Week 5', 'Week 6', 'Week 7', 'Week 8'],
    'Banda Sakti': [150, 160, 155, 165, 170, 175, 180, 185],
    'Blang Mangat': [200, 210, 205, 220, 225, 230, 235, 240],
    'Muara Satu': [130, 135, 140, 145, 150, 155, 160, 165],
    'Muara Dua': [180, 185, 190, 195, 200, 205, 210, 215],
    'Suhu (X1)': [20, 21, 19, 22, 20, 23, 24, 22],  # Faktor Eksogen 1
    'Harga Listrik per Watt (X2)': [5.5, 5.4, 5.6, 5.5,
                                    5.7, 5.8, 5.9, 5.6]  # Faktor Eksogen 2
}

# Membuat DataFrame
df = pd.DataFrame(data)

# Tampilkan DataFrame
display(df)


,Minggu,Banda Sakti,Blang Mangat,Muara Satu,Muara Dua,Suhu (X1),Harga Listrik per Watt (X2)
0,Week 1,150,200,130,180,20,5.5
1,Week 2,160,210,135,185,21,5.4
2,Week 3,155,205,140,190,19,5.6
3,Week 4,165,220,145,195,22,5.5
4,Week 5,170,225,150,200,20,5.7
5,Week 6,175,230,155,205,23,5.8
6,Week 7,180,235,160,210,24,5.9
7,Week 8,185,240,165,215,22,5.6


In [12]:
# Daftar wilayah
regions = ['Banda Sakti', 'Blang Mangat', 'Muara Satu', 'Muara Dua']

# Menyimpan DataFrame per wilayah
df_regions = {}

# Metrik kesalahan per wilayah
error_metrics = {}

# Definisikan warna untuk model
colors = {
    'Pemakaian Asli': 'blue',
    'MA (3-Mingguan)': 'orange',
    'AR(1)': 'green',
    'AR+MA': 'red',
    'VARMA': 'purple'
}

for region in regions:
    # Membuat salinan DataFrame untuk setiap wilayah
    df_region = df[['Minggu', region, 'Suhu (X1)', 'Harga Listrik per Watt (X2)']].copy()
    df_region.rename(columns={region: 'Y'}, inplace=True)
    
    # 1. Menghitung Moving Average (MA) 3-mingguan
    def moving_average(series, period=3):
        return series.rolling(window=period).mean()
    
    df_region['MA (3-Mingguan)'] = moving_average(df_region['Y'])
    
    # 2. Menghitung model Autoregressive (AR)
    phi_ar = 0.8  # Nilai phi untuk model AR(1)
    df_region['AR(1)'] = df_region['Y'].shift(1) * phi_ar
    
    # 3. Menghitung model AR + MA
    theta_ma = 0.2  # Nilai theta untuk model MA(1)
    df_region['Residuals'] = df_region['Y'] - df_region['MA (3-Mingguan)']
    df_region['Residuals'].fillna(0, inplace=True)  # Mengganti NaN dengan 0
    
    df_region['AR+MA'] = df_region['AR(1)'] + theta_ma * df_region['Residuals'].shift(1)
    df_region['AR+MA'].fillna(0, inplace=True)  # Mengganti NaN dengan 0
    
    # 4. Menghitung model VARMA(1,1)
    phi_varma = 1.0298
    theta_varma = 4.445
    beta1 = 0.3
    beta2 = 0.2
    
    def calculate_varma_values(series, residuals, phi, theta, x1, x2, beta1, beta2):
        varma_values = [None, None]  # VARMA tidak dapat dihitung untuk 2 minggu pertama
        for t in range(2, len(series)):
            varma_value = (phi * series[t-1] + theta * residuals[t-1] +
                           beta1 * x1[t-1] + beta2 * x2[t-1])
            varma_values.append(varma_value)
        return varma_values
    
    df_region['VARMA'] = calculate_varma_values(
        df_region['Y'], df_region['Residuals'], phi_varma, theta_varma,
        df_region['Suhu (X1)'], df_region['Harga Listrik per Watt (X2)'], beta1, beta2)
    
    # 5. Menghitung MAPE, RMSE, dan MAE
    error_metrics_region = {}
    models = ['MA (3-Mingguan)', 'AR(1)', 'AR+MA', 'VARMA']
    for model in models:
        actual = df_region['Y'][df_region[model].notna()]
        predicted = df_region[model][df_region[model].notna()]
        mae = mean_absolute_error(actual, predicted)
        rmse = mean_squared_error(actual, predicted, squared=False)
        mape = np.mean(np.abs((actual - predicted) / actual)) * 100
        error_metrics_region[model] = {'MAE': round(mae, 2), 'RMSE': round(rmse, 2), 'MAPE': round(mape, 2)}
    error_metrics[region] = error_metrics_region
    
    # Menyimpan df_region
    df_regions[region] = df_region.copy()

# Menampilkan metrik kesalahan untuk setiap wilayah
for region in regions:
    print(f"\nMetrik Kesalahan untuk {region}:")
    error_df_region = pd.DataFrame(error_metrics[region]).T
    display(error_df_region)



Metrik Kesalahan untuk Banda Sakti:


,MAE,RMSE,MAPE
MA (3-Mingguan),4.44,4.91,2.55
AR(1),38.00,38.28,22.33
AR+MA,51.46,63.66,31.73
VARMA,24.46,27.18,14.04



Metrik Kesalahan untuk Blang Mangat:


,MAE,RMSE,MAPE
MA (3-Mingguan),5.28,5.97,2.31
AR(1),49.29,49.59,22.03
AR+MA,67.46,84.17,31.49
VARMA,29.61,33.41,12.97



Metrik Kesalahan untuk Muara Satu:


,MAE,RMSE,MAPE
MA (3-Mingguan),5.00,5.00,3.29
AR(1),34.00,34.06,22.68
AR+MA,45.38,55.54,31.94
VARMA,25.50,26.90,16.54



Metrik Kesalahan untuk Muara Dua:


,MAE,RMSE,MAPE
MA (3-Mingguan),5.00,5.00,2.47
AR(1),44.00,44.05,22.01
AR+MA,60.38,75.44,31.45
VARMA,26.99,28.31,13.22


In [13]:
# 7. Plotting grafik perbandingan per wilayah dalam grid 2x2 menggunakan Plotly
fig = make_subplots(rows=2, cols=2, subplot_titles=regions)

for idx, region in enumerate(regions):
    df_region = df_regions[region]
    row = idx // 2 + 1
    col = idx % 2 + 1
    
    # Pemakaian Asli
    fig.add_trace(go.Scatter(x=df_region['Minggu'], y=df_region['Y'], mode='lines+markers', 
                             name='Pemakaian Asli', line=dict(color=colors['Pemakaian Asli'])), row=row, col=col)
    
    # Menambahkan model dengan warna yang ditentukan
    for model in models:
        show_legend = (region == regions[0])  # Hanya tunjukkan legend pada subplot pertama
        fig.add_trace(go.Scatter(x=df_region['Minggu'], y=df_region[model], mode='lines+markers', 
                                 name=model if show_legend else '', line=dict(color=colors[model]), showlegend=show_legend), row=row, col=col)

# Menyesuaikan layout
fig.update_layout(title_text='Perbandingan Pemakaian Listrik per Wilayah', height=800)
fig.update_xaxes(title_text='Minggu')
fig.update_yaxes(title_text='Pemakaian Listrik (mWh)')
fig.show()


In [14]:
# Menggabungkan semua wilayah dalam satu DataFrame untuk ditampilkan
combined_results = pd.DataFrame(columns=['Minggu', 'Wilayah', 'Pemakaian Asli', 'MA (3-Mingguan)', 'AR(1)', 'AR+MA', 'VARMA'])

for region in regions:
    df_region = df_regions[region][['Minggu', 'Y', 'MA (3-Mingguan)', 'AR(1)', 'AR+MA', 'VARMA']].copy()
    df_region['Wilayah'] = region
    df_region.rename(columns={'Y': 'Pemakaian Asli'}, inplace=True)
    combined_results = pd.concat([combined_results, df_region], ignore_index=True)

# Mengatur format desimal
combined_results = combined_results.round(2)

# Menampilkan DataFrame gabungan
display(combined_results)


C:\Users\User\AppData\Local\Temp\ipykernel_14716\3671013850.py:8: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



,Minggu,Wilayah,Pemakaian Asli,MA (3-Mingguan),AR(1),AR+MA,VARMA
0,Week 1,Banda Sakti,150,NaN,NaN,0.00,NaN
1,Week 2,Banda Sakti,160,NaN,120.0,120.00,NaN
2,Week 3,Banda Sakti,155,155.00,128.0,128.00,172.15
3,Week 4,Banda Sakti,165,160.00,124.0,124.00,166.44
4,Week 5,Banda Sakti,170,163.33,132.0,133.00,199.84
5,Week 6,Banda Sakti,175,170.00,136.0,137.33,211.84
6,Week 7,Banda Sakti,180,175.00,140.0,141.00,210.50
7,Week 8,Banda Sakti,185,180.00,144.0,145.00,215.97
8,Week 1,Blang Mangat,200,NaN,NaN,0.00,NaN
9,Week 2,Blang Mangat,210,NaN,160.0,160.00,NaN
